In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

In [20]:
# tfrecord 文件格式
# -> tf.train.Example
#     -> tf.train.Features -> {"key":tf.train.Features }
#         -> tf.train.Feature -> tf.train.ByteList/FloatList/Int64List

favarite_books =[name.encode('utf8')
                 for name in ['red and black','machine learning']]
favorite_books_bytelist = tf.train.BytesList(value= favarite_books)
print(favorite_books_bytelist)
hours_floatlist = tf.train.FloatList(value = [1.3,2.5,3.,4.6,4.9,5.0])
print(hours_floatlist)
age_int64list = tf.train.Int64List(value = [42])
print(age_int64list)
features = tf.train.Features(
        feature = {
            "favarite_books":tf.train.Feature(
                    bytes_list = favorite_books_bytelist ),
            "hours":tf.train.Feature(
                    float_list = hours_floatlist),
            "age":tf.train.Feature(
                    int64_list = age_int64list)         
                }
            )
print(features)

value: "red and black"
value: "machine learning"

value: 1.2999999523162842
value: 2.5
value: 3.0
value: 4.599999904632568
value: 4.900000095367432
value: 5.0

value: 42

feature {
  key: "age"
  value {
    int64_list {
      value: 42
    }
  }
}
feature {
  key: "favarite_books"
  value {
    bytes_list {
      value: "red and black"
      value: "machine learning"
    }
  }
}
feature {
  key: "hours"
  value {
    float_list {
      value: 1.2999999523162842
      value: 2.5
      value: 3.0
      value: 4.599999904632568
      value: 4.900000095367432
      value: 5.0
    }
  }
}



In [21]:
example = tf.train.Example(features = features)
print(example)
serialized_example = example.SerializeToString()
print(serialized_example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "favarite_books"
    value {
      bytes_list {
        value: "red and black"
        value: "machine learning"
      }
    }
  }
  feature {
    key: "hours"
    value {
      float_list {
        value: 1.2999999523162842
        value: 2.5
        value: 3.0
        value: 4.599999904632568
        value: 4.900000095367432
        value: 5.0
      }
    }
  }
}

b'\nl\n5\n\x0efavarite_books\x12#\n!\n\rred and black\n\x10machine learning\n%\n\x05hours\x12\x1c\x12\x1a\n\x18ff\xa6?\x00\x00 @\x00\x00@@33\x93@\xcd\xcc\x9c@\x00\x00\xa0@\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*'


In [22]:
output_dir = 'data/tfrecord_basic'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
filename = 'test.tfrecords'
filename_fullpath = os.path.join(output_dir, filename)
with tf.io.TFRecordWriter(filename_fullpath) as w:
    for i in range(3):
        w.write(serialized_example)

In [23]:
dataset = tf.data.TFRecordDataset([filename_fullpath])
for serialized_example_tensot in dataset:
    print(serialized_example_tensot)

tf.Tensor(b'\nl\n5\n\x0efavarite_books\x12#\n!\n\rred and black\n\x10machine learning\n%\n\x05hours\x12\x1c\x12\x1a\n\x18ff\xa6?\x00\x00 @\x00\x00@@33\x93@\xcd\xcc\x9c@\x00\x00\xa0@\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)
tf.Tensor(b'\nl\n5\n\x0efavarite_books\x12#\n!\n\rred and black\n\x10machine learning\n%\n\x05hours\x12\x1c\x12\x1a\n\x18ff\xa6?\x00\x00 @\x00\x00@@33\x93@\xcd\xcc\x9c@\x00\x00\xa0@\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)
tf.Tensor(b'\nl\n5\n\x0efavarite_books\x12#\n!\n\rred and black\n\x10machine learning\n%\n\x05hours\x12\x1c\x12\x1a\n\x18ff\xa6?\x00\x00 @\x00\x00@@33\x93@\xcd\xcc\x9c@\x00\x00\xa0@\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*', shape=(), dtype=string)


In [24]:
# 读取前定义数据类型，解析
expected_features = {
            'favorate_books': tf.io.VarLenFeature(dtype=tf.string),
            'hours': tf.io.VarLenFeature(dtype=tf.float32),
            'age':tf.io.FixedLenFeature([],dtype=tf.int64)
}
dataset = tf.data.TFRecordDataset([filename_fullpath])
for serialized_example_tensot in dataset:
    example = tf.io.parse_single_example(
            serialized_example_tensot,
            expected_features)
    books = tf.sparse.to_dense(example["favorate_books"],
                              default_value = b"")#对sparse进行解
    
    for book in books:
        print(book.numpy().decode("utf8"))


In [12]:
filename_fullpath_zip = filename_fullpath +'.zip'
options = tf.io.TFRecordOptions(compression_type='GZIP')
with tf.io.TFRecordWriter(filename_fullpath_zip) as w:
    for i in range(3):
        w.write(serialized_example)

In [19]:
dataset_zip = tf.data.TFRecordDataset([filename_fullpath_zip],
                                          compression_type='GZIP')
for serialized_example_tensot in dataset_zip:
    example = tf.io.parse_single_example(
            serialized_example_tensot,
            expected_features)
    books = tf.sparse.to_dense(example["favorate_books"],
                              default_value = b"")#对sparse进行解
    
    for book in books:
        print(book.numpy().decode("utf8"))

DataLossError: inflate() failed with error -3: incorrect header check [Op:IteratorGetNextSync]